### COMP - 472
Assignment Two

AI GURUS: James Partsafas, Ghaith Chrit, Samuel Collette

---------

#### Task 1

1) Add imports

In [1]:
import os
import gc
import csv
import random
import matplotlib.pyplot as plt
import pandas as pd
import gensim.downloader as api
from gensim.models import KeyedVectors
print("Imports complete")

Imports complete


2) Load model

In [2]:
print("starting model load")
model_name = 'word2vec-google-news-300'
model = KeyedVectors.load_word2vec_format(api.load(model_name, return_path=True), binary=True) 
print("model loaded")

starting model load
model loaded


3) Load test dataset

In [3]:
dataset_path = 'A2-DataSet/synonym.csv'
df = pd.read_csv(dataset_path)
print("Dataset loaded")

Dataset loaded


4) Run the model on test data and report result

In [4]:
if (not ('model_name' in locals())):
    model_name = ""
details_output_file = os.path.join('output', f'{model_name}-details.csv')
analysis_output_file = os.path.join('output', f'analysis.csv')

In [5]:
def create_details_file(details_output_file_name, df, model):
    correct_labels = 0
    answered_questions = 0
    with open(details_output_file_name, mode='w', newline='') as details_file:
      details_writer = csv.writer(details_file)
      details_writer.writerow(['question-word', 'correct-answer', 'guess-word', 'label'])

      for _, row in df.iterrows():
        question_word, correct_answer, guess_words = row['question'], row['answer'], row[2:].to_list()
        guess_words_in_vocab = [word for word in guess_words if word in model.index_to_key]

        if question_word in model.index_to_key and len(guess_words_in_vocab) > 0:
          most_similar_word = model.most_similar_to_given(question_word, guess_words_in_vocab)
          label = 'correct' if most_similar_word == correct_answer else 'wrong'
          correct_labels += 1 if label == 'correct' else 0
          answered_questions += 1
        else:
          label = 'guess'
          most_similar_word = random.choice(guess_words)

        details_writer.writerow([question_word, correct_answer, most_similar_word, label])
    
    return (correct_labels, answered_questions)

def write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, write_mode):
    accuracy = correct_labels / answered_questions if answered_questions > 0 else 0

    with open(analysis_output_file, mode=write_mode, newline='') as analysis_file:
      analysis_writer = csv.writer(analysis_file)
      if (write_mode == 'w'):  
          analysis_writer.writerow(['Model Name', 'Vocab Length', 'Number of Correct Labels', 'Number of Answered Questions', 'Accuracy of Answered Questions'])
      analysis_writer.writerow([model_name, len(model.index_to_key), correct_labels, answered_questions, accuracy])

In [6]:
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'w')

70 79


In [7]:
# Rid model to clear memory
model = None
gc.collect()

0

------------
#### Task 2

In [8]:
# First of 2 models with same embedding size
print("starting model load")
model_name = 'glove-twitter-50'
model = api.load(model_name)
print("model loaded")

starting model load
[==================================================] 100.0% 199.5/199.5MB downloaded
model loaded


In [9]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

36 78


In [10]:
model = None
gc.collect()

14

In [11]:
# Second of 2 models with same embedding size
print("starting model load")
model_name = 'glove-wiki-gigaword-50'
model = api.load(model_name)
print("model loaded")

starting model load
[==================================================] 100.0% 66.0/66.0MB downloaded
model loaded


In [12]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

57 80


In [13]:
model = None
gc.collect()

14

In [14]:
# First of 2 models with different embedding size
print("starting model load")
model_name = 'glove-twitter-25'
model = api.load(model_name)
print("model loaded")

starting model load
[==================================================] 100.0% 104.8/104.8MB downloaded
model loaded


In [15]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

36 78


In [16]:
model = None
gc.collect()

14

In [17]:
# Second of 2 models with different embedding size
print("starting model load")
model_name = 'glove-wiki-gigaword-100'
model = api.load(model_name)
print("model loaded")

starting model load
[==================================================] 100.0% 128.1/128.1MB downloaded
model loaded


In [18]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
correct_labels, answered_questions = create_details_file(details_output_file, df, model)
print(correct_labels, answered_questions)

write_to_analysis_file(analysis_output_file, correct_labels, answered_questions, 'a')

65 80


In [19]:
model = None
gc.collect()

14

In [74]:
def create_accuracy_bar_chart(models):
    analysis_file = 'output/analysis.csv'
    df_analysis = pd.read_csv(analysis_file)
    analysis_rows = df_analysis.loc[df_analysis['Model Name'].isin(models)]
    
    students_file = 'A2-DataSet/COMP-472-per-question.csv'
    df_students = pd.read_csv(students_file, encoding='UTF-16 LE')
    students_average_row = df_students.loc[0]
    
    accuracy_data = {'Baseline': 25, 'Students': students_average_row['Accuracy']}
    for _, row in analysis_rows.iterrows():
        accuracy_data[row['Model Name']] = row['Accuracy of Answered Questions'] * 100

    keys = list(accuracy_data.keys())
    values = list(accuracy_data.values())
    
    fig, ax = plt.subplots()
    plt.bar(keys, values)
    ax.tick_params(axis='x', labelrotation=90, labelsize=6)
    fig.patch.set_facecolor('white')
    
    plt.xlabel('Models')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy of different models')
    
    plt.tight_layout()
    plt.savefig('output/task2_accuracy.png', dpi=300)
    plt.close(fig)

In [75]:
create_accuracy_bar_chart(['word2vec-google-news-300', 'glove-twitter-50', 'glove-wiki-gigaword-50', 'glove-twitter-25', 'glove-wiki-gigaword-100'])

------------
#### Task 3

In [22]:
# TODO: Add code for task 3 here